In [33]:
import glob
import xml.etree.ElementTree as ET
import pandas as pd

In [56]:
filename = "../Observation Planner/exportall"

# first, get observation times for all targets
target_times = {}
for file in glob.glob("../Files/2024-04-01 Gemini Speckle Proposal/*.csv"):
    df = pd.read_csv(file)
    target_times = {
        **target_times,
        **{name: time for _, (name, time) in df[["Name", "Hours"]].iterrows()},
    }

# next, get the mapping of the nickname used in xml file to the TIC name
tree = ET.parse(f"{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[designator] = name

#finally, walk through each observation entry and add observing time to it
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = target_times[target_name]
    progTime = ET.Element("progTime")
    progTime.attrib["units"] = "hr"
    progTime.text = str(target_time)
    observation.append(progTime)
    partTime = ET.Element("partTime")
    partTime.attrib["units"] = "hr"
    partTime.text = "0.0"
    observation.append(partTime)
    time = ET.Element("time")
    time.attrib["units"] = "hr"
    time.text = str(target_time)
    observation.append(time)

tree.write(f"{filename}_new.xml")
